# Object Tracking with Avoidance 
**Full Coding At [Object_Tracking_and_Avoidance.py](Object_Tracking_and_Avoidance.py)** 
<br>
This is the documentation for Obstacle Avoidance (Without Camera)
<br>
*Library Used* 
- cv2
- time
- picamera2 
- libcamera 
- PCA9685_MC 
- Motor_Encoder 
- numpy as np
- threading
- Ultrasonic_sens
<br>



## Let's Start Coding ! 
### 1. Import all the Libraries 
- cv2
- time
- picamera2 
- libcamera 
- PCA9685_MC 
- Motor_Encoder 
- numpy as np
- threading
- Ultrasonic_sens

In [ ]:
import cv2
import time
from picamera2 import Picamera2
from libcamera import controls
from PCA9685_MC import Motor_Controller
from Motor_Encoder import Encoder
import numpy as np
import threading
from Ultrasonic_sens import Ultrasonic

### 2. Initialise all the variables and library 
- Initialise the camera library
  ```py 
  picam = Picamera2()
  ```
- Configure and start the camera
  ```py
  picam.configure(picam.create_preview_configuration(main={"format": 'XRGB8888', "size": (640, 480)}))
  picam.start()
  ```
- Set Autofocus to Continous mode ( This mode is only available on Raspberry Pi Camera Module 3 ) **Comment Out If not using RPi Camera Module 3**
    ```py
    picam.set_controls({"AfMode": controls.AfModeEnum.Continuous}) 
    ```

- Initialise the Motor library , encoder library and ultrasonic library 
  ```py
  Motor = Motor_Controller()
  enc = Encoder()
  ultrasonic = Ultrasonic()
  ```

-  Creates a lock to ensure only one thread can access the frame at a time. ( To Prevent the camera from crashing)
  ```py
  Frame_Lock = threading.Lock()
  ```

- Sets up a signal to stop all threads when triggered.
  ```py
  shutdown_event = threading.Event()
  ```

- Sets up a signal to switch to obstacle avoidance mode when triggered.
  ```py
  avoidance_event = threading.Event()
  ```
  
- Set up Global variables 
  - Vertical channel of the Pan Tilt Hat (Channel 0)
  - Horizontal channel of the Pan Tilt Hat (Channel 1)

- Move the Pan Tilt hat to the desire position 
  - Move the Pan Tilt hat in the Horizontal axis
  - Move the pan Tilt hat in the Vertical axis 

- Set Threshod value
  - Minimum area for to start the tracking 
    - MIN_AREA_THREASHOLD = 1500px 

In [ ]:
# Initialize camera, motor, encoder, and ultrasonic sensor
picam = Picamera2()
picam.configure(picam.create_preview_configuration(main={"format": 'XRGB8888', "size": (640, 480)}))
picam.start()
picam.set_controls({"AfMode": controls.AfModeEnum.Continuous})
Motor = Motor_Controller()
enc = Encoder()
ultrasonic = Ultrasonic()

# Threading synchronization
Frame_lock = threading.Lock()
shutdown_event = threading.Event() 
latest_frame = None

# Set initial servo position
vertical = 0
horizontal = 1
Motor.servoPulse(horizontal, 1250)
Motor.servoPulse(vertical, 1050)
# Thresholds
MIN_AREA_THRESHOLD = 1500  # Minimum area to detect color

### 3. The `camera` function
- This function is used to store the camera frame. 
- It will keep ruinning in the background 
- This function was masde to make sure that the camera is being called once to prevent double initialation error from the camera. 

In [ ]:
def camera():
    global latest_frame 
    picam.start() 
    while not shutdown_event.is_set():
        with Frame_lock:
            latest_frame = picam.capture_array()
        time.sleep(0.01) # Prevent high CPU usage
    picam.stop()

### 4. The `colorPicker` function 
- This function allows the user to select the target color they want to track in camera frame
- The user can adjust the HSV (Hue, Saturation, Value) values using a trackbar to fine-tune the color range.
- The goal is for the user to adjust the HSV values until only the desired object (target) is visible in the frame.
- The function creates a window that provides a live preview, enabling the user to visually adjust the color range in real-time.
- Once the desired color range is set, the function returns the **Lower Boundaries** and **Upper Boundaries** of the HSV color range for tracking purposes.
- Six trackbars are craeted to repreent the 
    - Lower Hue (l_h)
    - Lower Saturation(l_s)
    - Lower Value (l_v)
    - Upper Hue (u_h)
    - Upper Saturation (u_s)
    - Upper Value (u_v)
    
        ```py
        cv2.namedWindow("Color_Picker")
        cv2.createTrackbar("Lower Hue", "Color_Picker", 0, 179, nothing)
        cv2.createTrackbar("Lower Saturation", "Color_Picker", 0, 255, nothing)
        cv2.createTrackbar("Lower Value", "Color_Picker", 0, 255, nothing)
        cv2.createTrackbar("Upper Hue", "Color_Picker", 179, 179, nothing)
        cv2.createTrackbar("Upper Saturation", "Color_Picker", 255, 255, nothing)
        cv2.createTrackbar("Upper Value", "Color_Picker", 255, 255, nothing)
        ```
- When the function is called `While True` 
- Read the captured frame from the `camrera` function 
    ```py
    with Frame_lock:
            if latest_frame is not None:
                img = latest_frame.copy() # Copy the image from the camera thread
            else:
                continue # Skip the proces if no prame available 
    ```

- Convert the frame fromat from RGB to HSV format 
    ```py
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    ```

- Get the value from the trackbars 
    ```py
    l_h = cv2.getTrackbarPos("Lower Hue", "Color_Picker")
    l_s = cv2.getTrackbarPos("Lower Saturation", "Color_Picker")
    l_v = cv2.getTrackbarPos("Lower Value", "Color_Picker")
    u_h = cv2.getTrackbarPos("Upper Hue", "Color_Picker")
    u_s = cv2.getTrackbarPos("Upper Saturation", "Color_Picker")
    u_v = cv2.getTrackbarPos("Upper Value", "Color_Picker")
    ```
    
- Merge the value from the trackbars to get the ***lower_bound value** and **upper_bound value***
    ```py
    lower_bound = np.array([l_h, l_s, l_v])
    upper_bound = np.array([u_h, u_s, u_v])
    ```
- From them lower and upper bound value, filter out the other value from the frame, This only allow the adjusted value to be detected 
    ```py
    mask = cv2.inRange(hsv, lower_bound, upper_bound) 
    ``` 
- Placed the masked frame on the original frame to view the result 
    ```py 
    res = cv2.bitwise_and(img, img, mask=mask)
    ```
- convert the `mask` frame from GRAY (Black and White) format to BGRA (Colors) fromat
    - This step is needed to stack all the frames together. 
        ```py
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGRA)
        ```
- Stack the frame (image, res and mask) together 
    ```py
    stacked = np.hstack((mask, img, res))
    ```
- Show the trackbers and the frames in one windows (Color_Picker)
    ```py
    cv2.imshow("Color_Picker", cv2.resize(stacked, None, fx=0.4, fy=0.4))
    ```
- When the HSV value is set, User will required to press the `s` button to save the value
    - After the value saved, the Color_Picker window will be destroyed 
    - The function will be stop 
        ```py
        if cv2.waitKey(1) & 0xFF == ord('s'):
                cv2.destroyAllWindows()
                break
        ```
- At the end of the function, it wil return the lower_boundery and upper boundery value. 
    ```py
    return lower_bound, upper_bound
    ```


In [ ]:
def colorPicker():
    def nothing(x):
        pass

    cv2.namedWindow("Color_Picker")
    cv2.createTrackbar("Lower Hue", "Color_Picker", 0, 179, nothing)
    cv2.createTrackbar("Lower Saturation", "Color_Picker", 0, 255, nothing)
    cv2.createTrackbar("Lower Value", "Color_Picker", 0, 255, nothing)
    cv2.createTrackbar("Upper Hue", "Color_Picker", 179, 179, nothing)
    cv2.createTrackbar("Upper Saturation", "Color_Picker", 255, 255, nothing)
    cv2.createTrackbar("Upper Value", "Color_Picker", 255, 255, nothing)

    while True:
        with Frame_lock:
            if latest_frame is not None:
                img = latest_frame.copy() # Copy the image from the camera thread
            else:
                continue # Skip the proces if no prame available 
        
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        l_h = cv2.getTrackbarPos("Lower Hue", "Color_Picker")
        l_s = cv2.getTrackbarPos("Lower Saturation", "Color_Picker")
        l_v = cv2.getTrackbarPos("Lower Value", "Color_Picker")
        u_h = cv2.getTrackbarPos("Upper Hue", "Color_Picker")
        u_s = cv2.getTrackbarPos("Upper Saturation", "Color_Picker")
        u_v = cv2.getTrackbarPos("Upper Value", "Color_Picker")
        lower_bound = np.array([l_h, l_s, l_v])
        upper_bound = np.array([u_h, u_s, u_v])
        mask = cv2.inRange(hsv, lower_bound, upper_bound) 
        res = cv2.bitwise_and(img, img, mask=mask)
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGRA)
        stacked = np.hstack((mask, img, res))
        cv2.imshow("Color_Picker", cv2.resize(stacked, None, fx=0.4, fy=0.4))
        if cv2.waitKey(1) & 0xFF == ord('s'):
            cv2.destroyAllWindows()
            break
    return lower_bound, upper_bound


### 5. The `main` function

This function controls the color-based object tracking and obstacle avoidance for the robot. It uses HSV color space to detect an object and performs motor actions based on the object's position. If the object is not detected, the robot switches to obstacle avoidance mode using ultrasonic sensors. 

- Obtain HSV color boundaries from `colorPicker` function:
    - The program calls the `colorPicker()` function to get the **Lower Boundaries** and **Upper Boundaries** for HSV color space.
    - This is used to filter the target color for object detection.

        ```py
        lower_bound, upper_bound = colorPicker()
        cv2.destroyAllWindows()  # Close the color picker window after values are selected
        ```

- when the `shutdown_event` is not set:
    - A `while` loop is used to continuously process frames and check for the object as long as the `shutdown_event` is not triggered.

        ```py
        while not shutdown_event.is_set():
        ```

- Capture and process the frame:
    - A lock (`Frame_lock`) is used to ensure the frame is safely copied from the camera thread.
    - If no frame is available, the loop continues to wait for the next frame.
    - The frame is then converted from BGR to HSV color space for further processing.

        ```py
        with Frame_lock:
            if latest_frame is not None:
                img = latest_frame.copy()
            else:
                continue
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        ```

- Filter the frame based on HSV bounds:
    - The HSV image is filtered using the `lower_bound` and `upper_bound` obtained from the color picker to create a **mask**.
    - This mask isolates the target color by showing it as white and filtering out all other colors.

    ```py
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    ```

- Detect contours (objects):
    - The program uses the mask to find contours, which represent the boundaries of the detected object.
    - If no contours are found, the robot switches to obstacle avoidance mode:

        ```py
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        object_detected = False
        if contours: 
        ```

- Process detected contours:
    - For each detected contour, the program calculates the area.
    - If the contour area is greater than a minimum threshold (`MIN_AREA_THRESHOLD`), the object is considered detected.
    - A bounding box is drawn around the object, and the center point of the object is calculated.
    - Information such as the object's coordinates and area are displayed on the screen:

        ```py
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > MIN_AREA_THRESHOLD:
                object_detected = True
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.circle(img, (x + w // 2, y + h // 2), 5, (0, 0, 255), -1)
                cv2.putText(img, "Object", (x, y - 10), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(img, "Area: " + str(area), (x, y + h + 10), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(img, "Center of Object: (" + str(x + w // 2) + ", " + str(y + h // 2) + ")", (x, y + h + 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
        ```

- Motor control based on object position:
    - The robot’s movement is controlled based on the position of the object’s center along the X and Y axes.
    - Conditions for motor control:
        - **Y-axis** (forward/backward): If the Y coordinate is between 80 and 440, the robot moves.
        - **X-axis** (left/right): If the X coordinate is between 50 and 320, the robot turns left. Between 400 and 600, the robot turns right. If centered (320 to 400), the robot moves forward.
        - Otherwise, the robot brakes:

        ```py
        center_x = int(x + w // 2)
        center_y = int(y + h // 2)
        
        if 80 < center_y < 440:
            if 50 < center_x < 320:
                print("Turn Left")
                Motor.AntiClock_Rotate(20)
            if 400 < center_x < 600:
                print("Turn Right")
                Motor.Clock_Rotate(20)
            if 320 <= center_x <= 400:
                Motor.Forward(20)
                print("Centered")
            else:
                Motor.Brake()
        else:
            object_detected = False
        ```

- Switch to obstacle avoidance mode if no object detected:
    - If no object is detected, the robot switches to obstacle avoidance mode using ultrasonic sensors.
    - The ultrasonic sensors detect distances in three directions (left, front, and right).
    - Based on these distances, the robot will rotate or move backward to avoid obstacles:

        ```py
        if left and front and right is not None:
            if front < threshold or left < threshold or right < threshold:
                if front < threshold:
                    if front <= min_thresh_dist:
                        Motor.Backward(Speed)
                    elif left < min_thresh_dist and right < min_thresh_dist:
                        Motor.Backward(Speed)
                    elif left < threshold:
                        Motor.Clock_Rotate(rotation_speed)
                    elif right < threshold:
                        Motor.AntiClock_Rotate(rotation_speed)
                    else:
                        Motor.Backward(Speed)
                elif left < threshold:
                    Motor.Clock_Rotate(rotation_speed)
                elif right < threshold:
                    Motor.AntiClock_Rotate(rotation_speed)
            else:
                Motor.Forward(Speed)
        ```

- Display the result:
    - The processed frames (with the bounding box, contours, and motor instructions) are displayed in an OpenCV window:

        ```py
        cv2.imshow("Tracking", img)
        ```

- When the buton `q` is pressed:
    - The program checks if the user presses the **'q'** key, in which case the robot will stop all operations, clean up resources, and exit:
    
    ```py
    if cv2.waitKey(1) == ord('q'):
        print("Shutting down...")
        cv2.destroyAllWindows()
        Motor.cleanup()
        enc.stop()
        picam.stop()
        print("Program Terminated \n Exiting....")
        break
    ```

In [ ]:
# Main function where object tracking and obstacle avoidance happen sequentially
def main():
    lower_bound, upper_bound = colorPicker()  # Get color bounds from the color picker
    cv2.destroyAllWindows()

    while not shutdown_event.is_set():
        # 1. Run color tracker to check for the object
        with Frame_lock:
            if latest_frame is not None:
                img = latest_frame.copy()
            else:
                continue # Skip if no frame available

        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_bound, upper_bound)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        object_detected = False
        if contours:
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > MIN_AREA_THRESHOLD:
                    object_detected = True  # Object detected
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.circle(img, (x + w // 2, y + h // 2), 5, (0, 0, 255), -1)
                    cv2.putText(img, "Object", (x, y - 10), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.putText(img, "Area: " + str(area), (x, y + h + 10), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.putText(img, "Center of Object: (" + str(x + w // 2) + ", " + str(y + h // 2) + ")", (x, y + h + 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 2)

                    center_x = int(x + w // 2)
                    center_y = int(y + h // 2)

                    # Motor control based on object's center position
                    if 80 < center_y < 440:
                        if 50 < center_x < 320:
                            print("Turn Left")
                            Motor.AntiClock_Rotate(20)
                        if 400 < center_x < 600:
                            print("Turn Right")
                            Motor.Clock_Rotate(20)
                        if 320 <= center_x <= 400:
                            Motor.Forward(20)
                            print("Centered")
                        else:
                            Motor.Brake()
                    else:
                        object_detected = False
                else:
                    object_detected = False
                    cv2.imshow("Tracking", img)
                    break  # Exit after detecting and processing one object

        elif not object_detected and (not contours or area < MIN_AREA_THRESHOLD):
            print("No object detected, switching to obstacle avoidance.")
            cv2.putText(img, "Obstacle Avoidance Mode", (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 0, 0), 2)
            Speed = 20
            rotation_speed = 15
            threshold = 30
            min_thresh_dist = 10

            left, front, right = ultrasonic.distances()
            if left and front and right is not None:
                if front < threshold or left < threshold or right < threshold:
                    if front < threshold:
                        if front <= min_thresh_dist:
                            Motor.Backward(Speed)
                        elif left < min_thresh_dist and right < min_thresh_dist:
                            Motor.Backward(Speed)
                        elif left < threshold:
                            Motor.Clock_Rotate(rotation_speed)
                        elif right < threshold:
                            Motor.AntiClock_Rotate(rotation_speed)
                        else:
                            Motor.Backward(Speed)
                    elif left < threshold:
                        Motor.Clock_Rotate(rotation_speed)
                    elif right < threshold:
                        Motor.AntiClock_Rotate(rotation_speed)
                else:
                    Motor.Forward(Speed)
            cv2.imshow("Obstacle Avoidance", img)

        # Check for user input to quit the program
        if cv2.waitKey(1) == ord('q'):
            print("Shutting down...")
            cv2.destroyAllWindows()
            Motor.cleanup()
            enc.stop()
            picam.stop()
            print("Program Terminated \n Exiting....")
            break

### 6. Wrapping Up 
- When the programe start run the main function and start the camera thread

    ```py
    try:
        if __name__ == '__main__': 
            camera_thread = threading.Thread(target=camera)
            camera_thread.start()
            main()
    ``` 
- When there is keyboard interupt (Ctrl + c)
    ```py
    except KeyboardInterrupt:
        print("KeyboardInterrupt")
    ```
- When the programe ends
    - set the shutdown_event
    - Stop the camera thread 
    - Destroy all the windows created 
    - Stop the Motors 
    - stop the encoders 
    - Deinitialise the camera 
    ```py 
    finally:
        shutdown_event.set()
        camera_thread.join()
        cv2.destroyAllWindows()
        Motor.cleanup()
        enc.stop()
        picam.stop()
        print("Program Terminated \n Exiting....")
    ``` 

In [ ]:
try:
    if __name__ == '__main__':
        camera_thread = threading.Thread(target=camera)
        camera_thread.start()
        main()
except KeyboardInterrupt:
    print("KeyboardInterrupt")
finally:
    shutdown_event.set()
    camera_thread.join()
    cv2.destroyAllWindows()
    Motor.cleanup()
    enc.stop()
    picam.stop()
    print("Program Terminated \n Exiting....")
